In [1]:
# Copyright 2021 Annika Vogt
# Copyright 2021 ST John
# Copyright 2016 James Hensman, Nicolas Durrande
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


import numpy as np
from matplotlib import pyplot as plt
from scipy.cluster.vq import kmeans
from VFF.vgp import VGP_kron
import gpflow
from matplotlib import pyplot as plt
import tensorflow as tf

from gpflow.utilities import print_summary
gpflow.config.set_default_summary_fmt("notebook")

from time import time

In [2]:
X = np.loadtxt("titanic-x5.txt", delimiter="\t")
Y = np.loadtxt("titanic-y.txt")[:, None]


lik = gpflow.likelihoods.Bernoulli
k = gpflow.kernels.Matern32

a = X.min(0) - 2.
b = X.max(0) + 2.

In [3]:
models = []
labels = []

In [4]:
# Variational Fourier Features
for M in range(2,7):  
    m = VGP_kron(
        (X, Y), np.arange(M), a=a, b=b, kernels=[k()]*np.shape(X)[1], likelihood=lik(), use_two_krons=True
    )
    
    print_summary(m)
    models.append(m)
    labels.append(f"VFF {M}")
    
    # sparse
    Ts = [-9.210340,0.,0.295129,0.510786]
    ds = np.array([(M+1)**k for k in range(1,5)])
    
    for T,d in zip(Ts,ds):
        m = VGP_kron(
            (X, Y), np.arange(M), a=a, b=b, kernels=[k(), k(), k(), k(), k()], likelihood=lik(), use_two_krons=True, d=d, T=T, sparse=True
        )   
        models.append(m)
        labels.append(f"VFF {M} sparse. d = {d}, T = {T}")

name,class,transform,prior,trainable,shape,dtype,value
VGP_kron.kernels[0].variance VGP_kron.kernels[1].variance VGP_kron.kernels[2].variance VGP_kron.kernels[3].variance VGP_kron.kernels[4].variance,Parameter,Softplus,,True,(),float64,0.9999999999999999
VGP_kron.kernels[0].lengthscales VGP_kron.kernels[1].lengthscales VGP_kron.kernels[2].lengthscales VGP_kron.kernels[3].lengthscales VGP_kron.kernels[4].lengthscales,Parameter,Softplus,,True,(),float64,0.9999999999999999
VGP_kron.q_mu,Parameter,Identity,,True,"(243, 1)",float64,[[0....
VGP_kron.q_sqrt_kron[0],Parameter,Identity,,True,"(3, 3)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[1],Parameter,Identity,,True,"(3, 3)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[2],Parameter,Identity,,True,"(3, 3)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[3],Parameter,Identity,,True,"(3, 3)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[4],Parameter,Identity,,True,"(3, 3)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron_2[0],Parameter,Identity,,True,"(3, 3)",float64,"[[1.01, 0.01, 0.01..."
VGP_kron.q_sqrt_kron_2[1],Parameter,Identity,,True,"(3, 3)",float64,"[[1.01, 0.01, 0.01..."


name,class,transform,prior,trainable,shape,dtype,value
VGP_kron.kernels[0].variance VGP_kron.kernels[1].variance VGP_kron.kernels[2].variance VGP_kron.kernels[3].variance VGP_kron.kernels[4].variance,Parameter,Softplus,,True,(),float64,0.9999999999999999
VGP_kron.kernels[0].lengthscales VGP_kron.kernels[1].lengthscales VGP_kron.kernels[2].lengthscales VGP_kron.kernels[3].lengthscales VGP_kron.kernels[4].lengthscales,Parameter,Softplus,,True,(),float64,0.9999999999999999
VGP_kron.q_mu,Parameter,Identity,,True,"(3125, 1)",float64,[[0....
VGP_kron.q_sqrt_kron[0],Parameter,Identity,,True,"(5, 5)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[1],Parameter,Identity,,True,"(5, 5)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[2],Parameter,Identity,,True,"(5, 5)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[3],Parameter,Identity,,True,"(5, 5)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[4],Parameter,Identity,,True,"(5, 5)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron_2[0],Parameter,Identity,,True,"(5, 5)",float64,"[[1.01, 0.01, 0.01..."
VGP_kron.q_sqrt_kron_2[1],Parameter,Identity,,True,"(5, 5)",float64,"[[1.01, 0.01, 0.01..."


name,class,transform,prior,trainable,shape,dtype,value
VGP_kron.kernels[0].variance VGP_kron.kernels[1].variance VGP_kron.kernels[2].variance VGP_kron.kernels[3].variance VGP_kron.kernels[4].variance,Parameter,Softplus,,True,(),float64,0.9999999999999999
VGP_kron.kernels[0].lengthscales VGP_kron.kernels[1].lengthscales VGP_kron.kernels[2].lengthscales VGP_kron.kernels[3].lengthscales VGP_kron.kernels[4].lengthscales,Parameter,Softplus,,True,(),float64,0.9999999999999999
VGP_kron.q_mu,Parameter,Identity,,True,"(16807, 1)",float64,[[0....
VGP_kron.q_sqrt_kron[0],Parameter,Identity,,True,"(7, 7)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[1],Parameter,Identity,,True,"(7, 7)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[2],Parameter,Identity,,True,"(7, 7)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[3],Parameter,Identity,,True,"(7, 7)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[4],Parameter,Identity,,True,"(7, 7)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron_2[0],Parameter,Identity,,True,"(7, 7)",float64,"[[1.01, 0.01, 0.01..."
VGP_kron.q_sqrt_kron_2[1],Parameter,Identity,,True,"(7, 7)",float64,"[[1.01, 0.01, 0.01..."


name,class,transform,prior,trainable,shape,dtype,value
VGP_kron.kernels[0].variance VGP_kron.kernels[1].variance VGP_kron.kernels[2].variance VGP_kron.kernels[3].variance VGP_kron.kernels[4].variance,Parameter,Softplus,,True,(),float64,0.9999999999999999
VGP_kron.kernels[0].lengthscales VGP_kron.kernels[1].lengthscales VGP_kron.kernels[2].lengthscales VGP_kron.kernels[3].lengthscales VGP_kron.kernels[4].lengthscales,Parameter,Softplus,,True,(),float64,0.9999999999999999
VGP_kron.q_mu,Parameter,Identity,,True,"(59049, 1)",float64,[[0....
VGP_kron.q_sqrt_kron[0],Parameter,Identity,,True,"(9, 9)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[1],Parameter,Identity,,True,"(9, 9)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[2],Parameter,Identity,,True,"(9, 9)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[3],Parameter,Identity,,True,"(9, 9)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[4],Parameter,Identity,,True,"(9, 9)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron_2[0],Parameter,Identity,,True,"(9, 9)",float64,"[[1.01, 0.01, 0.01..."
VGP_kron.q_sqrt_kron_2[1],Parameter,Identity,,True,"(9, 9)",float64,"[[1.01, 0.01, 0.01..."


name,class,transform,prior,trainable,shape,dtype,value
VGP_kron.kernels[0].variance VGP_kron.kernels[1].variance VGP_kron.kernels[2].variance VGP_kron.kernels[3].variance VGP_kron.kernels[4].variance,Parameter,Softplus,,True,(),float64,0.9999999999999999
VGP_kron.kernels[0].lengthscales VGP_kron.kernels[1].lengthscales VGP_kron.kernels[2].lengthscales VGP_kron.kernels[3].lengthscales VGP_kron.kernels[4].lengthscales,Parameter,Softplus,,True,(),float64,0.9999999999999999
VGP_kron.q_mu,Parameter,Identity,,True,"(161051, 1)",float64,[[0....
VGP_kron.q_sqrt_kron[0],Parameter,Identity,,True,"(11, 11)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[1],Parameter,Identity,,True,"(11, 11)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[2],Parameter,Identity,,True,"(11, 11)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[3],Parameter,Identity,,True,"(11, 11)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron[4],Parameter,Identity,,True,"(11, 11)",float64,"[[1., 0., 0...."
VGP_kron.q_sqrt_kron_2[0],Parameter,Identity,,True,"(11, 11)",float64,"[[1.01, 0.01, 0.01..."
VGP_kron.q_sqrt_kron_2[1],Parameter,Identity,,True,"(11, 11)",float64,"[[1.01, 0.01, 0.01..."


In [5]:
for label, m in zip(labels, models):
    print(f"Optimizing {label}")
    start = time()
    try:
        o = gpflow.optimizers.Scipy()
        if isinstance(m, gpflow.models.ExternalDataTrainingLossMixin):
            loss = m.training_loss_closure((X, Y))
        else:
            loss = m.training_loss
        o.minimize(loss, m.trainable_variables)
    except:
        print("model optimization failed")
        
    print (round(time()-start), "s")
    print()

Optimizing VFF 2
11 s

Optimizing VFF 2 sparse. d = 3, T = -9.21034
9 s

Optimizing VFF 2 sparse. d = 9, T = 0.0
9 s

Optimizing VFF 2 sparse. d = 27, T = 0.295129
9 s

Optimizing VFF 2 sparse. d = 81, T = 0.510786
8 s

Optimizing VFF 3
8 s

Optimizing VFF 3 sparse. d = 4, T = -9.21034
11 s

Optimizing VFF 3 sparse. d = 16, T = 0.0
10 s

Optimizing VFF 3 sparse. d = 64, T = 0.295129
10 s

Optimizing VFF 3 sparse. d = 256, T = 0.510786
10 s

Optimizing VFF 4
9 s

Optimizing VFF 4 sparse. d = 5, T = -9.21034
20 s

Optimizing VFF 4 sparse. d = 25, T = 0.0
24 s

Optimizing VFF 4 sparse. d = 125, T = 0.295129
21 s

Optimizing VFF 4 sparse. d = 625, T = 0.510786
21 s

Optimizing VFF 5
12 s

Optimizing VFF 5 sparse. d = 6, T = -9.21034
20 s

Optimizing VFF 5 sparse. d = 36, T = 0.0
20 s

Optimizing VFF 5 sparse. d = 216, T = 0.295129
20 s

Optimizing VFF 5 sparse. d = 1296, T = 0.510786
54 s

Optimizing VFF 6
20 s

Optimizing VFF 6 sparse. d = 7, T = -9.21034
35 s

Optimizing VFF 6 sparse. d 

In [6]:
fp = open("Titanic Preciseness.txt", "w") 

for i, label, m in zip(range(np.size(labels)), labels, models):
    mu, var = m.predict_y(X)
    fp.write(label)
    fp.write(' %f' %(sum(np.equal(np.round(mu), Y)[:,0] * np.equal(np.round(mu), Y)[:,0])/Y.shape[0]))
    fp.write('\n')
    
fp.close() 

## 